# ME 592 Homework 4
## Jake Bergfeld, Mohammad Rashid Mohammad Shoaib, Melika Tajipour
#### Engineering Image Analysis - Distracted Driving Classification

##### Gathering data from Kaggle - Link to data: https://www.kaggle.com/competitions/state-farm-distracted-driver-detection/data

In [15]:
# !pip install kaggle
# !mkdir .kaggle  #naming required by kaggle API, creates a hidden folder
# !cp /home/exouser/Downloads/kaggle.json /home/exouser/.kaggle/kaggle.json

##### <u>Confirming location of Kaggle API token was moved successfully

In [21]:
# !cd .kaggle && ls

##### <u>Downloading the specific dataset and confirming locations

In [17]:
# !kaggle datasets list -s 'State Farm Distracted Driver Detection'

In [18]:
# !kaggle datasets download -d 'rightway11/state-farm-distracted-driver-detection'

In [19]:
# !sudo apt-get install unzip
# !unzip state-farm-distracted-driver-detection.zip -d data/

In [20]:
# !cd data && ls
# !cd data/imgs && ls
# !cd data/imgs/train && ls
# !cd data/imgs/test && ls

In [41]:
# Load driver details and image filenames
driver_imgs = pd.read_csv('/home/exouser/data/driver_imgs_list.csv')
driver_imgs

,subject,classname,img
0,p012,c0,img_10206.jpg
1,p012,c0,img_27079.jpg
2,p012,c0,img_50749.jpg
3,p012,c0,img_97089.jpg
4,p012,c0,img_37741.jpg
...,...,...,...
34919,p075,c9,img_15827.jpg
34920,p075,c9,img_16688.jpg
34921,p075,c9,img_64532.jpg
34922,p075,c9,img_7918.jpg


### <u>Data Information: 

### Default image size is 320x240
    
##### The 10 classes to predict are:
   - c0: normal driving
   - c1: texting - right
   - c2: talking on the phone - right
   - c3: texting - left
   - c4: talking on the phone - left
   - c5: operating the radio
   - c6: drinking
   - c7: reaching behind
   - c8: hair and makeup
   - c9: talking to passenger

##### *Formula to calculate the number of parameters in a CNN:*
   -  Convolutional layer: (in_channels x out_channels x kernel_height x kernel_width) + out_channels
   -  Batch normalization layer: 2 x num_features
   -  ReLU activation layer: 0 (no parameters)
   -  Max pooling layer: 0 (no parameters)
   -  Fully connected layer: (in_features x out_features) + out_features

### <u>Library Imports & Transforming Data

In [204]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import shutil
import random

In [205]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [206]:
#Transform the input images
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

### <u>Generating Test Data
#### Only needs to be run one time at the start

In [129]:
# #Only run this once to generate test and train data and create separate folders for each

# import os
# import random
# import shutil

# # Set the path to the 'Train & Test' folder
# folder_path = '/home/exouser/data/imgs/Train & Test'

# # Set the path to the 'Train' folder
# train_path = '/home/exouser/data/imgs/Train & Test/Train'

# # Set the path to the 'Test' folder
# test_path = '/home/exouser/data/imgs/Train & Test/Test'

# # Loop through each subfolder in the 'Train & Test' folder
# for subfolder in os.listdir(folder_path):

#     # Create a new subfolder in the 'Train' folder with the same name
#     train_subfolder = os.path.join(train_path, subfolder)
#     os.makedirs(train_subfolder, exist_ok=True)

#     # Create a new subfolder in the 'Test' folder with the same name
#     test_subfolder = os.path.join(test_path, subfolder)
#     os.makedirs(test_subfolder, exist_ok=True)

#     # List all the image files in the subfolder
#     images = os.listdir(os.path.join(folder_path, subfolder))
#     random.shuffle(images)

#     # Calculate the number of images to move to the 'Train' folder
#     split_index = int(len(images) * 0.7)

#     # Move the first 70% of images to the 'Train' folder
#     for image in images[:split_index]:
#         source = os.path.join(folder_path, subfolder, image)
#         destination = os.path.join(train_subfolder, image)
#         shutil.copyfile(source, destination)

#     # Move the remaining 30% of images to the 'Test' folder
#     for image in images[split_index:]:
#         source = os.path.join(folder_path, subfolder, image)
#         destination = os.path.join(test_subfolder, image)
#         shutil.copyfile(source, destination)


### <u>Dataloader and Pre-training

In [223]:
#Create a dataloader

#Directory path for the training & test images
train_path='/home/exouser/data/imgs/Train & Test/Train'
test_path='/home/exouser/data/imgs/Train & Test/Test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
) 


In [224]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)


['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


### <u>Step 1: Build and train a CNN model with roughly 500,000 parameters

In [225]:
#Building a CNN Network with ~500,000 parameters

class ConvNet500k(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet500k, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=1024)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=1024, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool(output)

        output = output.view(-1, 128 * 18 * 18)
        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)

        return output
        

In [226]:
model=ConvNet500k(num_classes=10).to(device)

In [227]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [228]:
num_epochs=10

In [229]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [230]:
print(train_count)
print(test_count)

12218
5244


In [231]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    # print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))

    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

Epoch: 0 Train Loss: tensor(3.9341) Train Accuracy: 0.29120969062039614 Test Accuracy: 0.526697177726926
Epoch: 1 Train Loss: tensor(0.8982) Train Accuracy: 0.7053527582255689 Test Accuracy: 0.8039664378337147
Epoch: 2 Train Loss: tensor(0.3272) Train Accuracy: 0.9048944180716975 Test Accuracy: 0.9445080091533181
Epoch: 3 Train Loss: tensor(0.1598) Train Accuracy: 0.9558029137338353 Test Accuracy: 0.9546147978642258
Epoch: 4 Train Loss: tensor(0.1043) Train Accuracy: 0.9686528073334425 Test Accuracy: 0.9649122807017544
Epoch: 5 Train Loss: tensor(0.0745) Train Accuracy: 0.9785562285153053 Test Accuracy: 0.9639588100686499
Epoch: 6 Train Loss: tensor(0.0567) Train Accuracy: 0.9842036339826485 Test Accuracy: 0.975209763539283
Epoch: 7 Train Loss: tensor(0.0474) Train Accuracy: 0.985922409559666 Test Accuracy: 0.8674675819984744
Epoch: 8 Train Loss: tensor(0.0507) Train Accuracy: 0.984449173350794 Test Accuracy: 0.975209763539283
Epoch: 9 Train Loss: tensor(0.0413) Train Accuracy: 0.98698

### <u>Now using unlabeled images to evaluate the network:

In [232]:
#Loading more libraries
import torch
import torch.nn as nn
import numpy as np
import torch.functional as F
import os
import pathlib
import glob
import cv2
from torch.autograd import Variable
from torchvision.transforms import transforms
from torchvision.models import squeezenet1_1
from io import open
from PIL import Image

In [233]:
pred_path='/home/exouser/data/imgs/Train & Test/Predict'

In [234]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet500k(num_classes=10)
model.load_state_dict(checkpoint)
model.eval()

ConvNet500k(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc1): Linear(in_features=41472, out_features=1024, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [235]:
#Transform the prediction input images
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [236]:
#Prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    image_tensor=transformer(image).float()
    image_tensore=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensors)
    
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    
    return pred

In [237]:
images_path=glob.glob(pred_path+'/*.jpg')

In [238]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [239]:
pred_dict

{}

### <u>Step 2: Train a model with roughly 10,000,000 parameters

In [195]:
class ConvNet10M(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet10M, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.relu1 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=128)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=256 * 18 * 18, out_features=4096)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=4096, out_features=2048)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(in_features=2048, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool(output)

        output = output.view(-1, 256 * 18 * 18)
        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)
        output = self.relu5(output)
        output = self.fc3(output)

        return output


In [196]:
model=ConvNet10M(num_classes=10).to(device)

In [197]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [198]:
num_epochs=10

In [199]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))

In [200]:
print(train_count)

12218


In [203]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    # print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))

    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

Epoch: 0 Train Loss: tensor(0.0886) Train Accuracy: 0.9719266655753805 Test Accuracy: 0.9471777269260107
Epoch: 1 Train Loss: tensor(0.0817) Train Accuracy: 0.9747912915370764 Test Accuracy: 0.9172387490465294
Epoch: 2 Train Loss: tensor(0.1097) Train Accuracy: 0.967670649860861 Test Accuracy: 0.9010297482837528
Epoch: 3 Train Loss: tensor(0.1112) Train Accuracy: 0.9638238664265837 Test Accuracy: 0.9584286803966438
Epoch: 4 Train Loss: tensor(0.0777) Train Accuracy: 0.9749549844491734 Test Accuracy: 0.9548054919908466
Epoch: 5 Train Loss: tensor(0.0873) Train Accuracy: 0.9728269765919135 Test Accuracy: 0.9719679633867276
Epoch: 6 Train Loss: tensor(0.0920) Train Accuracy: 0.9724995907677197 Test Accuracy: 0.9311594202898551
Epoch: 7 Train Loss: tensor(0.0899) Train Accuracy: 0.9716811262072352 Test Accuracy: 0.9553775743707094
Epoch: 8 Train Loss: tensor(0.0793) Train Accuracy: 0.9761826812899002 Test Accuracy: 0.9742562929061785
Epoch: 9 Train Loss: tensor(0.1013) Train Accuracy: 0.96